In [ ]:
from kafka import KafkaProducer
import threading
import json
import random
from time import sleep

from equipo.random_tweets import generate_random_tweets  

KAFKA_SERVER = 'a276c0c64413:9093'
TOPIC_PREFIX = 'tweet-'

In [ ]:
def worker(user_id: int, topic: str, num_messages: int):

    # Initialize the Kafka producer
    producer = KafkaProducer(
        bootstrap_servers=KAFKA_SERVER,
        value_serializer=lambda v: json.dumps(v).encode('utf-8')  # serialize data as JSON
    )

    print(f"[Thread-{user_id}] Iniciando, topic={topic}, mensajes={num_messages}")
    
    # Produce data to Kafka topic
    try:
        for tweet in generate_random_tweets(num_messages, user_id):
            # Send data to Kafka
            producer.send(topic, tweet)
            print(f"[Thread-{user_id}] Sent to {topic}: {tweet['tweet_id']}")

            sleep(3)

    except KeyboardInterrupt:
        print(f"[Thread-{user_id}] Interrupted.")
        
    finally:
        producer.close()
        print(f"[Thread-{user_id}] Producer closed.")

In [ ]:
if __name__ == "__main__":
    NUM_MESSAGES = random.randint(500, 501)
    threads = []

    for user_id in range(1, 5):
        topic = f"{TOPIC_PREFIX}{user_id}"
        t = threading.Thread(
            target=worker,
            args=(user_id, topic, NUM_MESSAGES),
            name=f"Thread-{user_id}"
        )
        threads.append(t)
        t.start()

    # Espera a que todos los hilos terminen
    for t in threads:
        t.join()

    print("Finished.")


[Thread-1] Iniciando, topic=tweet-1, mensajes=500
[Thread-4] Iniciando, topic=tweet-4, mensajes=500
[Thread-2] Iniciando, topic=tweet-2, mensajes=500
[Thread-3] Iniciando, topic=tweet-3, mensajes=500
[Thread-4] Sent to tweet-4: 1efc02da-3ad1-4c30-894f-af8987ee8870
[Thread-1] Sent to tweet-1: c39d300a-918f-4c7d-9990-9498500001cf
[Thread-2] Sent to tweet-2: 072b6e90-df40-4afe-8875-4861fed69d40
[Thread-3] Sent to tweet-3: 98588d0f-e4b6-475f-b95b-7c05a61c3a55
[Thread-4] Sent to tweet-4: 894eb56a-f6dc-4d40-977d-e1ce6a3239e3
[Thread-1] Sent to tweet-1: 722da950-cff1-41d5-b29d-f1c90cae93f9
[Thread-2] Sent to tweet-2: 72f85c63-c82a-42e2-bf11-7bd4fd46dd13
[Thread-3] Sent to tweet-3: b264f94b-915f-47ac-bd9a-21148bb82c83
[Thread-4] Sent to tweet-4: 5cfdf5bd-5ee2-4aec-956e-67afc9e50968
[Thread-1] Sent to tweet-1: e5e50a4f-f068-4558-b955-c6a1997ae894
[Thread-2] Sent to tweet-2: 4f8b2a4a-19d8-44aa-a5d4-10496bd34562
[Thread-3] Sent to tweet-3: 9b1e60f0-b8ce-4034-ba7f-22aa19f0ac2b
[Thread-4] Sent to t